### Quickstart: Compare runs, choose a model, and deploy it it a REST API

In this quickstart, we will:

- Run a hyperparameter sweep on the training script
- Compart the results of the runs in the MLflow UI
- Choose the best run and register it as a model UI
- Deploy the model to a REST API
- Build a container image suitable for deployment to a cloud platform.

In [1]:
import keras
import numpy as np
import pandas as pd

from hyperopt import STATUS_OK,Trials,fmin,hp,tpe

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [2]:
# Load the Dataset

data = pd.read_csv("F:/My_Design/Data_Science/Data Science/Data Science Notes/MLOPS/MLFLOW/MLFlow_Starter/2-DLProject/winequality-white.csv", sep=";")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
## Split the data into training validation and test sets

train,test = train_test_split(data,test_size=0.2,random_state=42)

In [4]:
train.shape,test.shape

((3918, 12), (980, 12))

In [13]:
train_x = train.drop(columns = ['quality'], axis=1)
train_y = train['quality']

In [14]:
train_x.shape, train_y.shape

((3918, 11), (3918,))

In [ ]:
# Test data
test_x = test.drop(columns=['quality'],axis=1)
test_y = test['quality']

In [16]:
test_x.shape, test_y.shape

((980, 11), (980,))

In [19]:
# Spliting the train data into train train and validation
X_train, X_val, y_train, y_val = train_test_split(train_x,train_y,test_size=0.2,random_state=42)

signature = infer_signature(X_train,y_train)

f:\My_Design\Data_Science\Data Science\Data Science Notes\MLOPS\MLFLOW\MLFlow_Starter\.venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


In [25]:
np.mean(X_train,axis=0)

fixed acidity             6.870868
volatile acidity          0.277891
citric acid               0.333854
residual sugar            6.360801
chlorides                 0.045511
free sulfur dioxide      34.908264
total sulfur dioxide    137.556637
density                   0.994030
pH                        3.190418
sulphates                 0.489722
alcohol                  10.520334
dtype: float64

In [27]:
keras.Input([X_train.shape[1]])

<KerasTensor shape=(None, 11), dtype=float32, sparse=False, ragged=False, name=keras_tensor>

In [ ]:
## Creating ANN Model

def train_model(params,epochs,X_train,y_train,X_val,y_val,test_x,test_y):

    ## Define the model architecture
    mean = np.mean(X_train,axis=0)
    var = np.var(X_train,axis=0)

    model = keras.Sequential(
        [
            keras.Input([X_train.shape[1]],name='Input_Layer'),
            keras.layers.Normalization(mean=mean,var=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
    )


    ## Compile the model

    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params['lr'],
        momentum=params['momentum']
    ),
    loss='mean_absolute_error',
    metrics=[keras.metrics.RootMeanSquaredError()]
    )


    ## Train the ANN model with lr and momentum params with MLflow tracking

    with mlflow.start_run(nested=True):
        model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=epochs,batch_size=32,verbose=0)

    ## Evaluate the model on test data
        eval_result=model.evaluate(X_val,y_val,batch_size=32,verbose=0)
        eval_rsme=eval_result[1]

    ## Log the model parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("val_rmse",eval_rsme)
        mlflow.keras.log_model(model,"model",signature=signature)

